In [1]:
# Define a list of colors for visualization
COLORS = [
    (255, 0, 0),  # Blue
    (0, 255, 0),  # Green
    (0, 0, 255),  # Red
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
    (255, 255, 255),  # White
    (0, 0, 0),  # Black
    # Add more colors as needed
]


# In[ ]:


import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("/home/greg/Desktop/dataset/yolov3.weights", "/home/greg/Desktop/dataset/yolov3.cfg")

# Load the classes
with open("/home/greg/Desktop/dataset/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Load image
img = cv2.imread("/home/greg/Downloads/cat.jpeg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)


# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Non-max suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Check if 'indices' is a list of lists (older OpenCV versions) or a flat array (newer versions)
if len(indices) > 0 and isinstance(indices[0], (list, np.ndarray)):
    indices = [i[0] for i in indices]  # Convert list of lists to flat list if necessary
else:
    # Ensure 'indices' is always a list even if it's empty or not a list of lists
    indices = indices.flatten().tolist()

for i in indices:
    box = boxes[i]
    x, y, w, h = box[0], box[1], box[2], box[3]
    if class_ids[i] < len(classes):
        label = str(classes[class_ids[i]])
        confidence = confidences[i]
        color = [int(c) for c in COLORS[class_ids[i] % len(COLORS)]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{label} {confidence:.2f}", (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    else:
        print(f"Warning: Class ID {class_ids[i]} is out of range. Skipping label.")

cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'cv2'